In [1]:
import pandas as pd
import numpy as np

In [11]:
data = pd.read_csv("/datos/data/Rodent_Inspection.csv")

In [12]:
data = data.sample(frac=0.01, replace=False, random_state=1)

In [13]:
data.shape

(20078, 20)

In [5]:
!pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [6]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, ParameterGrid
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.metrics import log_loss
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score
from time import time
import pandas as pd
import pickle


In [8]:
def feature_engineering(data):
    data = data.dropna()
    columns_to_drop = ["JOB_PROGRESS","BBL","BLOCK","LOT","HOUSE_NUMBER","STREET_NAME","X_COORD","Y_COORD","BOROUGH","LOCATION","APPROVED_DATE"]
    data = data.drop(columns_to_drop,axis=1)
    data.columns = ["job_ticket","Ins_type","job_id","boro_code","zc","lat","long","ins_date","result"]
    #data = data.drop(data[data.lat < 30].index)
    data.drop_duplicates()
    def conditions(s):
        if (s['result'] == "Bait applied") or (s['result'] == "Monitoring visit"):
            return 1
        else:
            return 0
    data['result'] = data.apply(conditions, axis=1)
    Insp = pd.get_dummies(data['Ins_type'])
    Insp=Insp.join(data.job_id)
    train = pd.merge(data.drop(['ins_date', 'Ins_type'], axis = 1),Insp, on="job_id")
    le = LabelEncoder()
    train['boro_code'] = le.fit_transform(train['boro_code'])
    train['result'] = le.fit_transform(train['result'])
    train['job_id'] = le.fit_transform(train['job_id'].astype(str))
      
    return train

In [14]:
data = data.dropna()
columns_to_drop = ["JOB_PROGRESS","BBL","BLOCK","LOT","HOUSE_NUMBER","STREET_NAME","X_COORD","Y_COORD","BOROUGH","LOCATION","APPROVED_DATE"]
data = data.drop(columns_to_drop,axis=1)
data.head()

,INSPECTION_TYPE,JOB_TICKET_OR_WORK_ORDER_ID,JOB_ID,BORO_CODE,ZIP_CODE,LATITUDE,LONGITUDE,INSPECTION_DATE,RESULT
1439152,Initial,12684688,PC7320179,3,11230.0,40.633968,-73.968414,11/26/2019 01:55:35 PM,Passed
844898,Initial,11888241,PC6597630,2,10474.0,40.815434,-73.887684,12/17/2012 12:26:55 PM,Passed
1732585,BAIT,2568008,PC6351184,1,10009.0,40.724435,-73.975990,05/17/2016 12:59:36 PM,Bait applied
841626,Initial,11941375,PC6148927,1,10027.0,40.808385,-73.942903,09/26/2013 03:02:23 PM,Passed
136061,Compliance,12959135,PC6796087,2,10467.0,40.873149,-73.875371,01/12/2017 02:12:59 PM,Rat Activity


In [15]:
data.columns = ["Ins_type","job_ticket","job_id","boro_code","zc","lat","long","ins_date","result"]
data = data.drop(data[data.lat < 30].index)
data.drop_duplicates()

,Ins_type,job_ticket,job_id,boro_code,zc,lat,long,ins_date,result
1439152,Initial,12684688,PC7320179,3,11230.0,40.633968,-73.968414,11/26/2019 01:55:35 PM,Passed
844898,Initial,11888241,PC6597630,2,10474.0,40.815434,-73.887684,12/17/2012 12:26:55 PM,Passed
1732585,BAIT,2568008,PC6351184,1,10009.0,40.724435,-73.975990,05/17/2016 12:59:36 PM,Bait applied
841626,Initial,11941375,PC6148927,1,10027.0,40.808385,-73.942903,09/26/2013 03:02:23 PM,Passed
136061,Compliance,12959135,PC6796087,2,10467.0,40.873149,-73.875371,01/12/2017 02:12:59 PM,Rat Activity
...,...,...,...,...,...,...,...,...,...
481303,Initial,11305633,PC7393223,4,11369.0,40.768005,-73.869442,07/25/2014 03:45:15 PM,Failed for Other R
1875664,BAIT,2767675,PC6499180,1,10025.0,40.801815,-73.966874,11/19/2019 11:45:29 AM,Bait applied
1467202,Initial,12257721,PC6242199,1,10027.0,40.814519,-73.953057,03/09/2017 11:12:02 AM,Passed
253244,Initial,11532471,PC6619626,2,10461.0,40.841294,-73.848571,03/12/2012 09:50:18 AM,Passed


In [16]:
def conditions(s):
    if (s['result'] == "Bait applied") or (s['result'] == "Monitoring visit"):
          return 1
    else:
          return 0
data['result'] = data.apply(conditions, axis=1)

In [17]:
Insp = pd.get_dummies(data['Ins_type'])
Insp=Insp.join(data.job_id)
train = pd.merge(data.drop(['ins_date', 'Ins_type'], axis = 1),Insp, on="job_id")

In [18]:
le = LabelEncoder()
train['boro_code'] = le.fit_transform(train['boro_code'])
train['result'] = le.fit_transform(train['result'])
train['job_id'] = le.fit_transform(train['job_id'].astype(str))
train['job_ticket'] = le.fit_transform(train['job_ticket'].astype(str))

NameError: name 'LabelEncoder' is not defined

In [ ]:
train=feature_engineering(data)

In [ ]:
    def modeling(data):
        X = data.drop(['results'], axis=1)
        y = data['results']
        #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12345)
        #modelos={}
      
      # xgboost
        model_xgb = xgb.XGBClassifier(
                                        n_estimators=10,
                                        max_depth=7,
                                        learning_rate=0.4,
                                        colsample_bytree=0.6,
                                        missing=-999,
                                        random_state=66
                                     )
        model_xgb.fit(X, y)
        pickle.dump(model_xgb, open('/content/drive/MyDrive/Rodent_inspection/data/entrenamiento_xgb.pkl', 'wb'))
 


        
      # logistic regression
        model_lr = LogisticRegression(
                                        penalty = 'l2',
                                        C= 1,
                                        solver='lbfgs'
                                      )
        model_lr.fit(X, y)
        pickle.dump(model_lr, open('/content/drive/MyDrive/Rodent_inspection/data/entrenamiento_lr.pkl', 'wb'))

      # KNN
        model_knn=KNeighborsClassifier(
                                        n_neighbors=10,
                                        weights='uniform',
                                        algorithm='auto'
                                      )
        model_knn.fit(X, y)
        pickle.dump(model_knn, open('/content/drive/MyDrive/Rodent_inspection/data/entrenamiento_knn.pkl', 'wb'))